# Imports

In [1]:
from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

from tensorflow.summary import create_file_writer

In [2]:
import os
import numpy as np

from time import strftime
from PIL import Image

# Constants

In [179]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS


nr_epochs = 70
learning_rate = 5e-4

n_hidden1 = 512
n_hidden2 = 64


# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 0 ns
Wall time: 12 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 5.86 s
Wall time: 5.86 s


In [8]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: total: 1.06 s
Wall time: 1.04 s


# Explore

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Loading and Preprocessing

In [14]:
# Re-scale features - aby sme dostali hodnoty 0 - 1 a nie 0 - 255, ako je to teraz
def preprocess_data(x, y):
    # Normalize images
    x = x / 255.0

    # One-hot encode labels
    y = tf.one_hot(y.astype(np.int32), depth=NR_CLASSES)
    return x, y

In [15]:
def get_data():
    # Load data from files
    x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=np.float32)
    y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=np.int32)
    x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=np.float32)
    y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=np.int32)
    
    # x -> sú obrázky v numpy
    # y -> sú správne čísla na nich

    # Validation dataset;
    x_val = x_train_all[:VALIDATION_SIZE]
    y_val = y_train_all[:VALIDATION_SIZE]

    x_train = x_train_all[VALIDATION_SIZE:]
    y_train = y_train_all[VALIDATION_SIZE:]

    # Preprocess the data
    x_train, y_train = preprocess_data(x_train, y_train)
    x_val, y_val = preprocess_data(x_val, y_val)
    x_test, y_test = preprocess_data(x_test, y_test)
    return x_train, y_train, x_val, y_val, x_test, y_test

# Tu reteštartovať

In [180]:
x_train, y_train, x_val, y_val, x_test, y_test = get_data()

# Setup Tensorflow Graph

In [181]:
# Folder for Tensorboard
folder_name = f'Model 1 at {strftime("%m %d %Y - %H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Succesfully created directory')


Succesfully created directory


# Tensorboard Setup

### Setting up writers ....

In [182]:
# Writer definition
train_writer = create_file_writer(directory + '/train')
val_writer = create_file_writer(directory + '/val')

In [183]:
# Model definition
# Model without dropout
class HandwritingModel(tf.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(HandwritingModel, self).__init__()
        # Initialize weights and biases for the first hidden layer
        self.w1 = tf.Variable(tf.random.truncated_normal([input_size, hidden1_size], stddev=0.1))
        self.b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))

        # Initialize weights and biases for the second hidden layer
        self.w2 = tf.Variable(tf.random.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))
        self.b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))

        # Initialize weights and biases for the output layer
        self.w3 = tf.Variable(tf.random.truncated_normal([hidden2_size, output_size], stddev=0.1))
        self.b3 = tf.Variable(tf.constant(0.1, shape=[output_size]))

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, TOTAL_INPUTS], dtype=tf.float32)])
    def __call__(self, x):
        # First hidden layer with ReLU activation
        with tf.name_scope('hidden_layer_1'):
            z1 = tf.matmul(x, self.w1) + self.b1
            a1 = tf.nn.relu(z1)

        # Second hidden layer with ReLU activation
        with tf.name_scope('hidden_layer_2'):
            z2 = tf.matmul(a1, self.w2) + self.b2
            a2 = tf.nn.relu(z2)

        # Output layer with linear activation
        with tf.name_scope('output_layer'):
            z3 = tf.matmul(a2, self.w3) + self.b3
        return z3

In [184]:
mod = HandwritingModel(TOTAL_INPUTS, n_hidden1, n_hidden2, NR_CLASSES)

In [185]:
"""# Model definition
# Model with dropout
class HandwritingModel_drop(tf.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(HandwritingModel_drop, self).__init__()
        # Initialize weights and biases for the first hidden layer
        self.w1 = tf.Variable(tf.random.truncated_normal([input_size, hidden1_size], stddev=0.1))
        self.b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))

        # Initialize weights and biases for the second hidden layer
        self.w2 = tf.Variable(tf.random.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))
        self.b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))

        # Initialize weights and biases for the output layer
        self.w3 = tf.Variable(tf.random.truncated_normal([hidden2_size, output_size], stddev=0.1))
        self.b3 = tf.Variable(tf.constant(0.1, shape=[output_size]))

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, TOTAL_INPUTS], dtype=tf.float32)])
    def __call__(self, x):
        # First hidden layer with ReLU activation
        with tf.name_scope('hidden_layer_1'):
            z1 = tf.matmul(x, self.w1) + self.b1
            a1 = tf.nn.relu(z1)

        # Drop hidden layer
        with tf.name_scope('drop_layer'):
            dz1 = tf.nn.dropout(a1, rate=0.8, name='dropout_layer')

        # Second hidden layer with ReLU activation
        with tf.name_scope('hidden_layer_2'):
            z2 = tf.matmul(dz1, self.w2) + self.b2
            a2 = tf.nn.relu(z2)

        # Output layer with linear activation
        with tf.name_scope('output_layer'):
            z3 = tf.matmul(a2, self.w3) + self.b3
        return z3"""

"# Model definition\n# Model with dropout\nclass HandwritingModel_drop(tf.Module):\n    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):\n        super(HandwritingModel_drop, self).__init__()\n        # Initialize weights and biases for the first hidden layer\n        self.w1 = tf.Variable(tf.random.truncated_normal([input_size, hidden1_size], stddev=0.1))\n        self.b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))\n\n        # Initialize weights and biases for the second hidden layer\n        self.w2 = tf.Variable(tf.random.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))\n        self.b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))\n\n        # Initialize weights and biases for the output layer\n        self.w3 = tf.Variable(tf.random.truncated_normal([hidden2_size, output_size], stddev=0.1))\n        self.b3 = tf.Variable(tf.constant(0.1, shape=[output_size]))\n\n    @tf.function(input_signature=[tf.TensorSpec(shape=[None, TOTAL

# Loss, Optimisation & Metrics

#### Defining Loss Function

In [186]:
def loss_fn(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

#### Defining Optimizer

In [187]:
# Optimizer
optimiser = tf.optimizers.Adam(learning_rate=learning_rate)

In [188]:
print(learning_rate)

0.0005


#### Accuracy Metric

In [189]:
# Create a accuracy metric
def accuracy_fn(predictions, labels):
    correct_predictions = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

#### Check Input Images in Tensorboard

In [190]:
#with tf.name_scope('show_image'):
#    x_image = tf.reshape(X, [-1, 28, 28, 1])
#    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [191]:
# Set-up učiacej slučky
size_of_batch = 1000
num_examples = y_train.shape[0]
print(num_examples)
nr_iterations = int(num_examples/size_of_batch)
print(nr_iterations)
index_in_epoch = 0

50000
50


#### Setup Filewriter and Merge Summaries

In [192]:
# nakreslenie tf grafu pre učiacu slučku .....
def pisac(model, x = "WDO"):
    # Get the concrete function
    concrete_function = model.__call__.get_concrete_function()

    # Create a summary file writer
    if x == "DO":
        writer = tf.summary.create_file_writer(directoryDO)
    else:
        writer = tf.summary.create_file_writer(directory)

    # Use the writer to write the graph
    with writer.as_default():
        tf.summary.graph(concrete_function.graph)
        writer.flush()

### Batching the Data

In [193]:
# Funkcia vráti batch z dát......
def get_batches(batch_size, x, y):
    for start in range(0, len(x), batch_size):
        end = start + batch_size
        yield x[start:end], y[start:end]

## Training Loop - Funkcia

In [194]:
def nauc (model):
    for epoch in range(nr_epochs):
        total_accuracy = tf.constant(0, dtype=tf.float32)
        num_batches = tf.constant(0, dtype=tf.float32)

        for batch_x, batch_y in get_batches(size_of_batch, x_train, y_train):
            with tf.GradientTape() as tape:
                # Vypočítame predpoved
                logits = model(batch_x)
                # Stratu
                loss = loss_fn(logits, batch_y)
                # úspešnosť
                accuracy = accuracy_fn(logits, batch_y)
                total_accuracy += accuracy

        # Compute gradients
        gradients = tape.gradient(loss, [model.w1, model.b1, model.w2, model.b2, model.w3, model.b3])
        if any(g is None for g in gradients):
            print(gradients)
            raise ValueError("One or more gradients are None")
            
        # Apply gradients
        optimiser.apply_gradients(zip(gradients, [model.w1, model.b1, model.w2, model.b2, model.w3, model.b3]))
        
        # Average accuracy
        average_train_accuracy = total_accuracy / num_batches
            
        with train_writer.as_default():
            for var, varname, grad in zip([model.w1, model.b1, model.w2, model.b2, model.w3, model.b3], ['w1', 'b1', 'w2', 'b2', 'w3', 'b3'], gradients):
                tf.summary.histogram(varname, var, step=epoch)
                tf.summary.histogram(f"{varname}/grad", grad, step=epoch)
            # Performance metrics
            with tf.name_scope('performance'):
                tf.summary.scalar('accuracy', accuracy, step=epoch)
                tf.summary.scalar('cost', loss, step=epoch)

        num_batches += 1

        # Validation
        val_logits = model(tf.cast(x_val, tf.float32))
        val_loss = loss_fn(val_logits, y_val)
        val_accuracy = accuracy_fn(val_logits, y_val)
        
        with val_writer.as_default():
            with tf.name_scope('performance'):
                tf.summary.scalar('accuracy', val_accuracy, step=epoch)
                tf.summary.scalar('cost', val_loss, step=epoch)
        
        print(f"Epoch {epoch + 1}, Loss (validation): {val_loss.numpy()}, Training Accuracy: {average_train_accuracy.numpy() * 100:.20f}%, Validation Accuracy: {val_accuracy.numpy() * 100:.2f}%")

        train_writer.flush()
        val_writer.flush()

    # Napíše graf modelu......
    pisac(model)

## A teraz natrenovanie...

In [195]:
# Model bez Dropoff
nauc(mod)

Epoch 1, Loss (validation): 2.4630229473114014, Training Accuracy: inf%, Validation Accuracy: 5.94%
Epoch 2, Loss (validation): 2.3123090267181396, Training Accuracy: inf%, Validation Accuracy: 12.26%
Epoch 3, Loss (validation): 2.1967904567718506, Training Accuracy: inf%, Validation Accuracy: 20.56%
Epoch 4, Loss (validation): 2.1015095710754395, Training Accuracy: inf%, Validation Accuracy: 28.01%
Epoch 5, Loss (validation): 2.018244504928589, Training Accuracy: inf%, Validation Accuracy: 35.70%
Epoch 6, Loss (validation): 1.9416017532348633, Training Accuracy: inf%, Validation Accuracy: 42.06%
Epoch 7, Loss (validation): 1.8679312467575073, Training Accuracy: inf%, Validation Accuracy: 47.54%
Epoch 8, Loss (validation): 1.7950366735458374, Training Accuracy: inf%, Validation Accuracy: 51.95%
Epoch 9, Loss (validation): 1.721903920173645, Training Accuracy: inf%, Validation Accuracy: 55.74%
Epoch 10, Loss (validation): 1.6482328176498413, Training Accuracy: inf%, Validation Accuracy:

# Make a Prediction

In [196]:
img = Image.open('MNIST/test_img.png')
img

In [197]:
# Konverzia do ČB obrázka
bw = img.convert('L')

In [198]:
img_array = np.invert(bw)

In [199]:
img_array.shape

(28, 28)

In [200]:
test_img = img_array.ravel()

In [201]:
test_img.shape

(784,)

In [202]:
# Reshape the test_img to match the expected shape (None, 784)
test_img = test_img.reshape(1, -1)

In [203]:
predictions = tf.argmax(mod(test_img), axis=1)

In [204]:
print(f'Prediction for test image is {predictions}')

Prediction for test image is [2]


### Teraz všetky obrázky.....

In [213]:
image_directory = 'MNIST/'
processed_images = []
preds = []

def tfprocimg(obr):
    img = Image.open(obr)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel()
    test_img = test_img.reshape(1, -1)
    return test_img

In [218]:
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Check if the file is a JPEG or PNG image (you can add more formats as needed)
        file_path = os.path.join(image_directory, filename)
        
        try:
            # Open and process the image using Pillow
            ts_im = tfprocimg(file_path)
            predict = tf.argmax(mod(ts_im), axis=1)
            print(f'File: {file_path} is number {predict}')
            
            # Append the processed image to the list and preds
            processed_images.append(file_path)
            preds.append(predict)
            
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

File: MNIST/test_img.png is number [2]
File: MNIST/test_img1.png is number [9]
File: MNIST/test_img1a.png is number [0]
File: MNIST/test_img6.png is number [6]
File: MNIST/test_img7.png is number [2]
File: MNIST/test_img7a.png is number [1]
File: MNIST/test_img7b.png is number [1]
File: MNIST/test_img9.png is number [3]


# Testing and Evaluation

**Challenge:** Calculate the accuracy over the test dataset (```x_test``` and ```y_test```). Use your knowledge of running a session to get the accuracy. Display the accuracy as a percentage rounded to two decimal numbers. 

In [207]:
# Make predictions on the test data
predictions = mod(x_test)

# Calculate accuracy
test_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.argmax(y_test, axis=1)), tf.float32))

# Convert accuracy to a percentage and round to two decimal places
accuracy_percent = tf.round(test_accuracy * 100, 2)

# Print the accuracy as a percentage
print(f"Accuracy: {accuracy_percent}%")
                          
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy: 83.0%
Accuracy on test set is 83.17%


# Reset for the Next Run

In [208]:
train_writer.close()
val_writer.close()
